In [1]:
'''正解データ（スランプ値が20.0）の動画はIOHD0017とする'''
import Function

sec_ranges = [(48, 53), (30, 35), (45, 50), (40, 45)]
Function.ExtractFramesFromMultipleVideos(sec_ranges)

In [2]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt

try:
    # 画像の数字データを格納する配列
    x_data = []
    # 正解ラベル
    y_data = []
    # 入力データの形状
    input_shape = [256, 256, 3]
    # 学習データ画像が保存されているディレクトリパス
    image_dir = '../data/Movie'

    # "IOHD"で始まるディレクトリを探索
    for sub_dir in sorted(os.listdir(image_dir)):
        sub_dir_path = os.path.join(image_dir, sub_dir)
        if os.path.isdir(sub_dir_path) and sub_dir.startswith("IOHD"):
            # 各サブディレクトリ内のjpgファイルを読み込む
            for filename in sorted(os.listdir(sub_dir_path)):
                if filename.endswith(".jpg"):
                    img_path = os.path.join(sub_dir_path, filename)
                    img = load_img(img_path, target_size=input_shape)
                    img_array = img_to_array(img) / 255.0
                    x_data.append(img_array)
                    if sub_dir == "IOHD0017":
                        y_data.append(True)
                    else:
                        y_data.append(False)


    # 正規化
    x_data = np.array(x_data, dtype="float32")
    
    # 確認
    print(x_data.shape)
    print(len(y_data))

except Exception as e:
    print(f"エラー発生：{e}")

2024-09-30 17:20:39.624162: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-30 17:20:39.640665: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-30 17:20:39.646088: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-30 17:20:39.659266: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-30 17:20:40.716715: W tensorflow/compiler/tf2

(1198, 256, 256, 3)
1198


In [6]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import optimizers

def make_convlayer():
    model = Sequential()

    # 第1層：畳み込み層1
    # (バッチサイズ, 32, 3, 3) -> (バッチサイズ, 32, 32, 32)
    model.add(
        Conv2D(
            filters=32,
            kernel_size=(3,3),
            input_shape=x_train[0].shape,
            padding="same",
            activation="relu"
        ))
    # 第2層：畳み込み層2
    # (バッチサイズ, 32, 32, 32) -> (バッチサイズ, 32, 32, 32)
    model.add(
        Conv2D(
            filters=32,
            kernel_size=(3,3),
            padding="same",
            activation="relu"
        ))
    # 第3層：プーリング層1
    # (バッチサイズ, 32, 32, 32) -> (バッチサイズ, 16, 16, 32)
    model.add(MaxPooling2D(pool_size=(2,2)))
    # ドロップアウト1
    model.add(Dropout(0.5))
    # 第4層：畳み込み層3
    # (バッチサイズ, 16, 16, 32) -> (バッチサイズ, 16, 16, 64)
    model.add(
        Conv2D(
            filters=64,
            kernel_size=(3,3),
            padding="same",
            activation="relu"
        ))
    # 第5層：畳み込み層4
    # (バッチサイズ, 16, 16, 64) -> (バッチサイズ, 16, 16, 64)
    model.add(
        Conv2D(
            filters=64,
            kernel_size=(3,3),
            padding="same",
            activation="relu"
        ))
    # 第6層：プーリング層2
    # (バッチサイズ, 16, 16, 64) -> (バッチサイズ, 8, 8, 64)
    model.add(MaxPooling2D(pool_size=(2,2)))
    # ドロップアウト2
    model.add(Dropout(0.5))
    # Flatten：4階テンソルから2階テンソルに変換
    # (バッチサイズ, 8, 8, 64) -> (バッチサイズ, 4096)
    model.add(Flatten())
    # 第7層：全結合層
    # (バッチサイズ, 4096) -> (バッチサイズ, 512)
    model.add(Dense(512,
                    activation="relu"))
    # ドロップアウト3
    model.add(Dropout(0.5))
    # 第8層：出力層
    # (バッチサイズ, 512) -> (バッチサイズ, 10)
    model.add(Dense(10,
                    activation="softmax"))
    
    # Sequentialオブジェクトのコンパイル
    model.compile(
        loss="categorical_crossentropy",
        optimizer=optimizers.Adam(0.001),
        metrics=["accuracy"]
    )

    return model